In [ ]:
!pip install transformers
!pip install umap-learn
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.3 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 99.9 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 304 kB/s 
     |████████████████████████████████| 1.1 MB 9.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=b58930f5d82690679de984a5133ff1047ae8ac323ae3e9556b7675053bb95073
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pyn

In [ ]:
from unittest.util import _MAX_LENGTH
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer
from transformers import BertModel
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch.optim import Adam
from tqdm import tqdm
from sklearn.manifold import TSNE
import plotly.express as px
from torch.nn.functional import normalize
from sklearn.decomposition import PCA
import random


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
config={
    'batch': 8, 
    'epochs': 10, # 총 10이 최적인듯. 
    'lr' : 1e-6

}
names=['text','emotion','crap']
val_df = pd.read_csv('drive/MyDrive/dev.tsv', sep='\t',names=names)
test_df = pd.read_csv('drive/MyDrive/test.tsv', sep='\t',names=names)
train_df = pd.read_csv('drive/MyDrive/train.tsv', sep='\t',names=names)

test_df=test_df.drop(columns=['crap'])
val_df=val_df.drop(columns=['crap'])
train_df=train_df.drop(columns=['crap'])

pos_idx=['1','15','17','18']  
neg_idx=['2','11','14','25']
mask_idx=pos_idx + neg_idx

idx_map={'1': 0, '15': 1,'17': 2, '18': 3, '2': 4, '11': 5,'14': 6, '25': 7 } 
emotion_map=['amusement', 'gratitude', 'love', 'joy','fear','anger','disgust','sadness']

comma_idx=train_df[train_df['emotion'].str.contains(',')].index
train_df=train_df.drop(comma_idx)
comma_idx=test_df[test_df['emotion'].str.contains(',')].index
test_df=test_df.drop(comma_idx)
comma_idx=val_df[val_df['emotion'].str.contains(',')].index
val_df=val_df.drop(comma_idx)

train_df=train_df[ train_df['emotion'].isin(mask_idx) ]
val_df=val_df[ val_df['emotion'].isin(mask_idx) ]
test_df=test_df[ test_df['emotion'].isin(mask_idx) ] #총 26713개

train_df=train_df.reset_index(drop=True)
test_df=test_df.reset_index(drop=True)
val_df=val_df.reset_index(drop=True)

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer,idx_map):
        self.label = df['emotion']
        self.text = [tokenizer(text, padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]
        self.idx_map=idx_map
        self.sentence=df['text']
   
    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx): 
        str_label=self.label[idx]
        return self.text[idx], self.idx_map[str_label], self.sentence[idx]



class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 8) 
        self.relu = nn.ReLU()

    def forward(self, input_id, mask): 

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output) 

        return final_layer, dropout_output


def train(model, train_df, val_df, tokenizer, config, idx_map):
    trainDataset, valDataset = Dataset(train_df,tokenizer,idx_map), Dataset(val_df,tokenizer,idx_map)
    train_dataloader = DataLoader(trainDataset, batch_size=config['batch'], shuffle=True) 
    val_dataloader = DataLoader(valDataset, batch_size=config['batch'], shuffle=True)
    batch=config['batch']

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    learning_rate=config['lr']
    epochs=config['epochs']
    batch=config['batch']

    loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    last_val=0
   
    if use_cuda:

            model = model.cuda()
            loss = loss.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input,train_label,_ in tqdm(train_dataloader): 
                train_label=train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output, _ = model(input_id, mask) 
               
                batch_loss = loss(output, train_label)
                total_loss_train += batch_loss.item()
               
                acc = (output.argmax(dim=1) == train_label).sum().item() 
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
           
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label, _ in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    

                    output, _ = model(input_id, mask)

                    batch_loss = loss(output, val_label)
                    total_loss_val += batch_loss.item()
                   
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

                    
           
            val_accuracy=total_acc_val / len(val_df)
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_df): .3f} \
                | Train Accuracy: {total_acc_train / len(train_df): .3f} \
                | Val Loss: {total_loss_val / len(val_df): .3f} \
                | Val Accuracy: {total_acc_val / len(val_df): .3f}')
            
            if val_accuracy < last_val:
              break
            else:
              last_val=val_accuracy

    torch.save(model.state_dict(),'drive/MyDrive/go_final_model.pt')
    


                 
def test(model, test_df, tokenizer, idx_map,emotion_map):

    testDataset=Dataset(test_df,tokenizer,idx_map)

    test_dataloader = DataLoader(testDataset, batch_size=config['batch'])

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    vecs_df=pd.DataFrame([])
    with torch.no_grad():

        for i, (test_input, test_label, sentence) in enumerate(test_dataloader):

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output, pooled_output = model(input_id, mask)


              output_cpu=output.cpu()
              output_cpu=torch.softmax(output_cpu,dim=1)

              pooled_output_cpu=pooled_output.cpu()
              
              sentence_cpu=sentence
              labels_cpu=output_cpu.argmax(dim=1)
              emotion_cpu=[emotion_map[k] for k in labels_cpu]

              #embeddings_cpu=output_cpu.tolist()
              embeddings_cpu=pooled_output_cpu.tolist()

              embeddings_df=pd.DataFrame(embeddings_cpu)
              embeddings_df['label']=emotion_cpu
              embeddings_df['sentence']=sentence_cpu

              vecs_df=pd.concat((vecs_df,embeddings_df))
              vecs_df=vecs_df.reset_index(drop=True)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

              
    print(f'Test Accuracy: {total_acc_test / len(testDataset): .3f}')

    
    dim_to_reduce=768
    vectors=vecs_df.iloc[:,:dim_to_reduce].values
    reduced_vec=dim_reduc(vectors)
    reduced_vec=torch.tensor(reduced_vec)
    vecs_df['x']=reduced_vec[:,0]
    vecs_df['y']=reduced_vec[:,1]
    vecs_df=vecs_df.drop(columns=range(dim_to_reduce))
    
    return vecs_df






tokenizer = BertTokenizer.from_pretrained('bert-base-cased') 
model = BertClassifier()

#model.load_state_dict(torch.load('drive/MyDrive/go_final_model.pt'))
#torch.save(model, 'drive/MyDrive/model.pt')
train(model,train_df,val_df,tokenizer,config,idx_map)
test_vecs_df=test(model,test_df,tokenizer, idx_map,emotion_map)
past_visualize(test_vecs_df)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test Accuracy:  0.876


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

